In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import wrds
import datetime
import statsmodels.api as sm

In [2]:
db = wrds.Connection() 
#goqhuB-1hafqe-dojvix

Enter your WRDS username [max]:antb95
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
sp = db.get_table(library = 'crsp' , table = 'dsp500p')
start = np.where(sp.caldt >= datetime.date(2011,3,17))[0][0]
sp = sp.loc[start:,['caldt','spindx']]

In [4]:
names = db.get_table(library='crsp', table='stocknames')
names =names.loc[:,['permco' , 'ticker']]

In [5]:
Ticker = ['IBM','MCD','MMM','WMT']
Permco = np.zeros(len(Ticker))
for i in range(len(Ticker)):
    inter = names['permco'].where(names['ticker'] == Ticker[i])
    inter.dropna(inplace = True)
    Permco[i] = inter.iloc[1]

In [6]:
df = pd.DataFrame(index = np.arange(0,len(sp)),columns = ['Date' , 'SPX'] + Ticker)

In [7]:
df.Date = np.array(sp.caldt)
df.SPX = np.array(sp.spindx)
for i in range(len(Ticker)):
    inter = db.raw_sql("select prc, date from crsp.dsf where permco in ("+str(Permco[i])+") and date >='2011-03-17' and date <='2018-12-31'")
    df[Ticker[i]] = inter.prc

In [8]:
df.loc[:,df.columns != 'Date'] = np.log(df.loc[:,df.columns != 'Date']) - np.log(df.loc[:,df.columns != 'Date']).shift(1)
df.dropna(inplace = True)

In [47]:
X = pd.DataFrame({'Ones' : [1] * len(df), 'SPX' : np.array(df.SPX)},index = np.arange(1,len(df)+1,1))
Beta = pd.DataFrame(columns = Ticker,index = ['a' , 'Beta'])
Eps = pd.DataFrame(columns = Ticker, index = df.index)
for i in range(len(Ticker)):
    Y = df[Ticker[i]]
    model = sm.OLS(Y,X).fit()
    Beta.at[:,Ticker[i]] = model.params.to_list()
    Eps.at[:,Ticker[i]] = Y - np.matmul(np.array(X),np.array(Beta[Ticker[i]]))
Beta

,IBM,MCD,MMM,WMT
a,-0.000449,0.000254,0.000050,0.000112
Beta,0.848814,0.569866,0.984636,0.554722


In [51]:
Eps.corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.024981,0.007512,0.001233
MCD,0.024981,1.000000,0.013255,0.118189
MMM,0.007512,0.013255,1.000000,0.013046
WMT,0.001233,0.118189,0.013046,1.000000


In [61]:
df[Ticker].corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.347704,0.492350,0.282380
MCD,0.347704,1.000000,0.419919,0.327442
MMM,0.492350,0.419919,1.000000,0.358268
WMT,0.282380,0.327442,0.358268,1.000000


In [12]:
df.SPX

1       0.004293
2       0.014882
3      -0.003557
4       0.002910
5       0.009297
          ...   
1956   -0.027487
1957    0.048403
1958    0.008526
1959   -0.001242
1960    0.008457
Name: SPX, Length: 1960, dtype: float64